In [ ]:
import sys
#!{sys.executable} -m pip install snorkel-metal
#!{sys.executable} -m pip install tensorboardX
#!{sys.executable} -m pip install git+https://github.com/HazyResearch/snorkel
#!{sys.executable} -m pip install sqlalchemy
#!{sys.executable} -m pip install matplotlib
#!{sys.executable} -m pip install spacy
#!{sys.executable} -m pip install lxml
#!{sys.executable} -m pip install treedlib
#!{sys.executable} -m pip install numba
#!{sys.executable} -m pip install nltk

## Preprocessing

In [89]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

import re
import pickle
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.metrics import confusion_matrix, f1_score

from metal.label_model import LabelModel
from metal.analysis import lf_summary, label_coverage
from metal.label_model.baselines import MajorityLabelVoter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
import pandas as pd
df = pd.read_csv("../data/interim/kenya-positive.csv")
df_negative = pd.read_csv("../data/interim/kenya-neutral.csv")
df = df.append(df_negative, ignore_index = True)


df['label'] = range(0, len(df))
df = df[['label', 'sentences', 'class']]
df.reset_index()
df.to_csv('../data/interim/kenya.tsv', sep = '\t', index = False)

In [164]:
df['class'] = df['class'].map({'n' : 0, 'i' : 1, 'd' : 0})

In [ ]:
[x for x in df['sentences'][df['class'] == 1]]

## Creating labeling functions

In [166]:
incentive = {'fund established', 'market connection', 'low transaction cost',
            'microfinance', 'payment system', 'financial support', 'grants',
            'funded by', 'provide funding', 'support', 'bilateral', 'multilateral',
            'capital', 'concessional', 'results based', 'mobilize', 'philanthropic',
            'finance', 'credit', 'carbon market', 'provide investment oppportunities',
            'offer incentives', 'generated funds', 'qualify for grants',
            'will be funded', 'budgetary allocation', 'credit packages'}

positive_words = {'established', 'created', 'improve', 'provide', 'mobilize', 'increase',
                 'access to', 'implement'}

negative_words = {'failed', 'there is not', 'there has not been', 'inadequate',
                 'obstacles', 'barriers', 'obstacles'}

disincentive = {'high transaction costs', 'barriers', 'levied', 'evasion', 
                'penalty', 'excise', 'seizure', 'fee', 'disincentive'}

# Look for postitive words before incentives or the lack of negative words before incentives


In [175]:
ABSTAIN = 0 
POSITIVE = 1
NEGATIVE = 2
            
INCENTIVE = r"""\b(fund established|market connection|low transaction cost| microfinance|payment system|financial support|grants|
                    funded by|provide funding|support|bilateral|multilateral|capital|concessional|results based|mobilize|philanthropic|
                    finance|fund|credit|carbon market|provide investment opportunities|offer incentives|generated funds|
                    qualify for grants|will be funded|budgetary allocation|credit packages|increase investment|
                    carbon credit|transfer pricing|mobilization of financ|encourage investment|facilitate investment|
                    access to finance|increased revenue|improving the lending|market access)"""

def contains_positive_expressions(text):
    return POSITIVE if re.search(INCENTIVE, text) else ABSTAIN

LFs = [
    contains_positive_expressions,
]

LF_names = [
    'positive',
]

In [176]:
def make_Ls_matrix(data, LFs):
    noisy_labels = np.empty((len(data), len(LFs)))
    for i, row in data.iteritems():
        for j, lf in enumerate(LFs):
            noisy_labels[i][j] = lf(row.lower())
    return noisy_labels

In [177]:
LF_matrix = make_Ls_matrix(df['sentences'], LFs)
gold_standard = np.array(df['class'])
LF_matrix = LF_matrix.reshape((432,))

confusion_matrix(LF_matrix, gold_standard)

array([[298,  13],
       [ 57,  64]])

In [178]:
f1_score(LF_matrix, gold_standard)

0.6464646464646465